In [10]:
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from math import pi, ceil
plt.style.use('seaborn')
sns.set_style("whitegrid")

import urllib.request
from nltk.tokenize import RegexpTokenizer

import re
import random
import numpy as np
import pandas as pd
import random
import math
import itertools
import multiprocessing
from tqdm import tqdm
import logging
import pickle

from scipy.sparse.linalg import svds,eigs
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models.callbacks import CallbackAny2Vec
from scipy import stats

from time import time
import warnings
warnings.filterwarnings(action='ignore')

import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GIUCO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
track = pd.read_csv('tracks.csv')
track = pd.DataFrame(track)
print(track.shape)
track.head()

(3222, 22)


,Unnamed: 0,id,artist_name,track_name,album_name,artist_genre,release_date,artist_popularity,track_popularity,artist_followers,...,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,3nqQXoyQOWXiESFLlDF1hG,Sam Smith,Unholy (feat. Kim Petras),Unholy (feat. Kim Petras),"['dance pop', 'pop', 'uk pop']",2022-09-22,89,99,21138669,...,2,-7.375,0.0864,0.0130,0.000005,0.2660,0.238,131.121,156943,4
1,1,1Qrg8KqiBpW07V7PNxwwwL,SZA,Kill Bill,SOS,"['pop', 'r&b']",2022-12-08,94,86,9525089,...,8,-5.747,0.0391,0.0521,0.144000,0.1610,0.418,88.980,153947,4
2,2,1PckUlxKqWQs3RlWXVBLw3,Lizzo,About Damn Time,About Damn Time,"['dance pop', 'escape room', 'minnesota hip ho...",2022-04-14,79,85,5263886,...,10,-6.305,0.0656,0.0995,0.000000,0.3350,0.722,108.966,191822,4
3,3,4k6Uh1HXdhtusDW5y8Gbvy,Steve Lacy,Bad Habit,Gemini Rights,['afrofuturism'],2022-07-15,85,87,2694509,...,1,-7.093,0.0355,0.6130,0.000058,0.4020,0.700,168.946,232067,4
4,4,7jHi6opPq439ruUlfH90Aw,Metro Boomin,Creepin' (with The Weeknd & 21 Savage),HEROES & VILLAINS,['rap'],2022-12-02,92,74,3457729,...,1,-5.985,0.0474,0.4100,0.000000,0.0784,0.152,97.950,221520,4


In [12]:
def cleanText(readData):
    text = re.sub('[-=+#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', readData)
 
    return text

In [13]:
genre = []
for i in track['artist_genre']:
    if i == '[]':
        i = 'NA'
        genre.append(i.strip())
    else:
        i = cleanText(i)
        genre.append(i.strip())
track['genre'] = genre


In [14]:
track = track[track['genre'] != "NA"]
track = track.reset_index()
track['track_popularity'] = track['track_popularity'] / 100 

In [15]:
track

,index,Unnamed: 0,id,artist_name,track_name,album_name,artist_genre,release_date,artist_popularity,track_popularity,...,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genre
0,0,0,3nqQXoyQOWXiESFLlDF1hG,Sam Smith,Unholy (feat. Kim Petras),Unholy (feat. Kim Petras),"['dance pop', 'pop', 'uk pop']",2022-09-22,89,0.99,...,-7.375,0.0864,0.0130,0.000005,0.2660,0.238,131.121,156943,4,"dance pop, pop, uk pop"
1,1,1,1Qrg8KqiBpW07V7PNxwwwL,SZA,Kill Bill,SOS,"['pop', 'r&b']",2022-12-08,94,0.86,...,-5.747,0.0391,0.0521,0.144000,0.1610,0.418,88.980,153947,4,"pop, rb"
2,2,2,1PckUlxKqWQs3RlWXVBLw3,Lizzo,About Damn Time,About Damn Time,"['dance pop', 'escape room', 'minnesota hip ho...",2022-04-14,79,0.85,...,-6.305,0.0656,0.0995,0.000000,0.3350,0.722,108.966,191822,4,"dance pop, escape room, minnesota hip hop, pop..."
3,3,3,4k6Uh1HXdhtusDW5y8Gbvy,Steve Lacy,Bad Habit,Gemini Rights,['afrofuturism'],2022-07-15,85,0.87,...,-7.093,0.0355,0.6130,0.000058,0.4020,0.700,168.946,232067,4,afrofuturism
4,4,4,7jHi6opPq439ruUlfH90Aw,Metro Boomin,Creepin' (with The Weeknd & 21 Savage),HEROES & VILLAINS,['rap'],2022-12-02,92,0.74,...,-5.985,0.0474,0.4100,0.000000,0.0784,0.152,97.950,221520,4,rap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,3216,3555,27qAMKrDdKEs8HDXcvR24R,Chris Rea,Driving Home for Christmas - 2019 Remaster,"Dancing with Strangers (Deluxe Edition, 2019 R...","['art rock', 'blues rock', 'classic rock', 'me...",1987-08-02,75,0.85,...,-10.863,0.0380,0.3550,0.000020,0.1800,0.866,180.183,241627,4,"art rock, blues rock, classic rock, mellow gol..."
2885,3217,3556,2TE4xW3ImvpltVU0cPcKUn,Shakin' Stevens,Merry Christmas Everyone,Merry Christmas Everyone,"['europop', 'new wave pop', 'rockabilly', 'sof...",1991-11-25,75,0.91,...,-3.315,0.0369,0.1290,0.000000,0.1280,0.960,202.899,220160,4,"europop, new wave pop, rockabilly, soft rock"
2886,3218,3557,26i4jKBlWSrlzpiB9V9gDB,Iuly Neamtu,Fata cu paharul,Fata cu paharul,['manele'],2022-03-04,55,0.54,...,-4.603,0.2100,0.0665,0.000000,0.1200,0.413,171.956,142166,4,manele
2887,3219,3558,6r2UqgPxaMxZEXwrOnZ7bm,YNY Sebi,Tarabanele,Most Wanted,['romanian trap'],2022-06-23,54,0.52,...,-3.464,0.0470,0.0139,0.000114,0.1350,0.528,91.015,139780,4,romanian trap


In [16]:
from ContentTFIDF import ContentTFIDF
ct = ContentTFIDF(track)
ct_tfidf = ct.calculateTFIDF()
ct_tfidf.shape


(2889, 2530)

In [17]:
from ContentBasedRecommender import ContentBasedRecommender

a = ContentBasedRecommender(track, ct_tfidf)
b = a.user_info()
c = a.recommend_features()
d = a.recommend_genre()
e = a.feature_genre_intersection(track, c,d)
f = a.get_total_score()
f

What's your favourite song out of these?   1) Teenagers  2) Anaconda  3) Parking Lots  4) I miss the old u  5) Walking Disaster
Light or dark song?   1) light  2) dark
Fast songs or slow songs?   1) fast  2) slow
What's your current mood?   1) happy  2) meh  3) sad


,artist_name,track_name,album_name,mood_score,speed_score,emotion_score,genre_score,total_score
1771,LMFAO,Party Rock Anthem,Sorry For Party Rocking,0.6480,0.596173,0.684723,0.873697,0.758331
830,will.i.am,Scream & Shout,#willpower,0.4990,0.596260,0.680199,0.873697,0.732758
1566,Black Eyed Peas,I Gotta Feeling,THE E.N.D. (THE ENERGY NEVER DIES),0.3900,0.586782,0.687653,0.873697,0.714254
310,Black Eyed Peas,Where Is The Love?,Elephunk,0.1910,0.431427,0.648093,0.873697,0.648602
2043,Gwen Stefani,Bubble Pop Electric,Love Angel Music Baby,0.6460,0.889481,0.749059,0.532166,0.646840
2651,Blxst,Keep Calling,Keep Calling,0.5130,0.421930,0.397907,0.847898,0.646089
998,PnB Rock,Selfish,GTTM: Goin Thru the Motions,0.9601,0.467629,0.598961,0.577931,0.626747
2358,Iggy Azalea,Emo Club Anthem,The End of an Era,0.8560,0.568724,0.537308,0.590486,0.622248
192,Chase Atlantic,Friends,Nostalgia,0.9089,0.492202,0.627400,0.546151,0.611159
1686,Don Toliver,Flocky Flocky (feat. Travis Scott),Life of a DON,0.8060,0.623576,0.574007,0.553292,0.610577
